# Data Import

In [1]:
import pandas as pd
import numpy as np

## Data

In [2]:
artists = pd.read_csv("../../../data/processed/artists.csv")
tracks = pd.read_json("../../../data/v2/tracks.json")
track_storage = pd.read_json("../../../data/v2/track_storage.json")
users = pd.read_json("../../../data/v2/users.json")

In [3]:
track_ids = tracks.id
VALID_COLUMN_NAMES = ['duration_ms', 'popularity', 'explicit', 'release_date','danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
tracks = tracks[VALID_COLUMN_NAMES]
rd = tracks.release_date
rd = pd.to_datetime(rd, errors='coerce')
tracks['release_date'] = rd.dt.year.fillna(0).astype(int)

In [12]:
users.head()

,user_id,favourite_genres,premium_user
0,101,"[dance pop, latin, hard rock]",False
1,102,"[reggaeton, latin arena pop, modern rock]",False
2,103,"[rap, art rock, rock]",False
3,104,"[mexican pop, contemporary country, psychedeli...",False
4,105,"[rock, adult standards, permanent wave]",False


In [4]:
users = users.drop(columns=['name', 'city', 'street'])
GENRES = np.unique(np.concatenate(users['favourite_genres'].to_numpy()))

In [6]:
one_hot = pd.get_dummies(tracks['key'], "key")
for x in range(16):
    if x not in tracks['key'].unique():
        one_hot[f'key_{x}'] = 0
one_hot.head()

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,key_12,key_13,key_14,key_15
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Model

Classifier input:
    1. Track Desccription (12 numerical + 2 categorical giving 16 one-hots + 1 binary)
    2. User Description (2 categorical giving 50 one-hots + 1 binary)
    3. Music Fingerprint (5 categorical giving 5x50 one-hots)
Classifier output:
    1. P(User will listen whole track)
    2. P(User will like the track)
    3. P(Track is similar to fingerprint)

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Music_classifier(nn.Module):
    def __init__(self, genres):
        self.genres = genres
        #    |
        # 50 | Genres | 10    }
        #    |                } USER [11]
        # czy_premium | 1     }
    
        #    |
        # 16 | key    | 3     }
        #    |                } TRACK [16]
        # numeric     | 12    }
        # binary      | 1     }
    
        # TRACK [16] |        }
        # TRACK [16] |   | 10 } FINGERPRINT [10]
        # TRACK [16] |        }
        track_key_code = 3
        user_genre_code = 10
        fingerprint_params = 15
        super(Music_classifier, self).__init__()
        ##### USER PREP ######
        self.emb_user_genre = nn.Linear(len(genres), user_genre_code)
        self.emb_user_genre_act = nn.LeakyReLU()

        ##### TRACK PREP #####
        self.emb_track_key = nn.Linear(16, track_key_code)
        self.emb_track_key_act = nn.LeakyReLU()

        ##### FINGERPRINT PREP #####
        self.emb_fingerprint = nn.Linear(3 * (track_key_code + 12 + 1), fingerprint_params)
        self.emb_fingerprint_act = nn.LeakyReLU()
        self.emb_fingerprint_track1 = nn.Linear(16, track_key_code)
        self.emb_fingerprint_track1_act = nn.LeakyReLU()
        self.emb_fingerprint_track2 = nn.Linear(16, track_key_code)
        self.emb_fingerprint_track2_act = nn.LeakyReLU()
        self.emb_fingerprint_track3 = nn.Linear(16, track_key_code)
        self.emb_fingerprint_track3_act = nn.LeakyReLU()

        #### MAIN CLASSIFIER ####
        user_params = user_genre_code + 1
        track_params = track_key_code + 12 + 1

        self.layers = nn.Sequential(
            nn.Linear(user_params + track_params + fingerprint_params, 256),
            nn.LeakyReLU(),

            nn.BatchNorm1d(256),
            nn.Linear(256, 128),
            nn.Dropout(0.1),
            nn.LeakyReLU(),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 3),

            nn.Sigmoid()
        )
    def forward(self, user, track, finger):
        # user -> [Tensor(50, bs), Tensor(1, bs)]
        # track -> [Tensor(16, bs), Tensor(13, bs)]
        # finger -> [ [Tensor(16, bs), Tensor(13, bs)], [Tensor(16, bs), Tensor(13, bs)], [Tensor(16, bs), Tensor(13, bs)] ]

        comp_emb_user_genre = self.emb_user_genre(user[0])
        comp_emb_user_genre = self.emb_user_genre_act(comp_emb_user_genre)

        comp_emb_track_key = self.emb_track_key(track[0])
        comp_emb_track_key = self.emb_track_key_act(comp_emb_track_key)

        comp_emb_fingerprint_track1 = self.emb_fingerprint_track1(finger[0][0])
        comp_emb_fingerprint_track1 = self.emb_fingerprint_track1_act(comp_emb_fingerprint_track1)
        comp_emb_fingerprint_track2 = self.emb_fingerprint_track2(finger[1][0])
        comp_emb_fingerprint_track2 = self.emb_fingerprint_track2_act(comp_emb_fingerprint_track2)
        comp_emb_fingerprint_track3 = self.emb_fingerprint_track3(finger[2][0])
        comp_emb_fingerprint_track3 = self.emb_fingerprint_track3_act(comp_emb_fingerprint_track3)
        emb_finger_x = torch.cat([comp_emb_fingerprint_track1, comp_emb_fingerprint_track2, comp_emb_fingerprint_track3], dim=1)
        comp_emb_fingerprint = self.emb_fingerprint(emb_finger_x)
        comp_emb_fingerprint = self.emb_fingerprint_act(comp_emb_fingerprint)

        x = torch.cat([comp_emb_user_genre, user[1], comp_emb_track_key, track[1], comp_emb_fingerprint])

        return self.layers(x)
